In [5]:
import pandas as pd
import warnings
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, JSON, DateTime
from datetime import datetime
warnings.filterwarnings("ignore")

In [6]:
engine = create_engine('sqlite:///database/user_presets.db')

# Your SQL query
query = "SELECT * FROM profiles"

# Execute the query and fetch the results into a Pandas DataFrame
df = pd.read_sql_query(query, engine)
df

,id,user_name,profile_name,settings,timestamp_UTC
0,1,Luke,Luke_test_,"{""time_line"": ""7 Days"", ""start_date"": ""2023-11...",2023-11-21 11:15:58.527492
1,2,Luke,Luke_test_2,"{""time_line"": ""Lifetime"", ""start_date"": ""2023-...",2023-11-21 11:23:56.229463
2,3,Alice,Alice_test_,"{""time_line"": ""7 Days"", ""start_date"": ""2023-10...",2023-11-21 11:26:19.325723
3,4,Alice,Alice_test_2,"{""time_line"": ""7 Days"", ""start_date"": ""2023-10...",2023-11-21 11:32:52.580235
4,5,Luke,Luke_test_3,"{""time_line"": ""Lifetime"", ""start_date"": ""2023-...",2023-11-21 11:36:21.295979
5,6,Luke,Luke_test_3,"{""time_line"": ""Lifetime"", ""start_date"": ""2023-...",2023-11-21 11:42:00.739298
6,7,Luke,Luke_test_4,"{""time_line"": ""7 Days"", ""start_date"": ""2023-11...",2023-11-21 11:46:29.421150
7,8,Alice,Alice_test_3,"{""time_line"": ""14 Days"", ""start_date"": ""2023-1...",2023-11-21 11:47:18.904856
8,9,Alice,Alice_test_5,"{""time_line"": ""14 Days"", ""start_date"": ""2023-1...",2023-11-21 11:47:51.617848
9,10,Alice,Alice_test_6,"{""time_line"": ""Lifetime"", ""start_date"": ""2023-...",2023-11-21 11:52:12.499596


In [7]:
Base = declarative_base()
class Profile(Base):
    """
    SQLAlchemy model for the 'profiles' table.

    Attributes:
        id (int): Unique identifier for each profile.
        user_name (str): User name associated with the profile.
        profile_name (str): Name of the profile.
        settings (dict): JSON representation of profile settings.
        timestamp_UTC (datetime): Timestamp indicating when the profile was created (in UTC).

    Note:
        The 'timestamp_UTC' field is automatically populated with the current UTC time when a new profile is created.

    Example:
        To create a new profile instance:
        >>> profile = Profile(user_name='JohnDoe', profile_name='Default', settings={'option1': True, 'option2': 'value'})
    """

    __tablename__ = 'profiles'

    id = Column(Integer, unique=True, primary_key=True)
    user_name = Column(String)
    profile_name = Column(String)
    settings = Column(JSON)
    timestamp_UTC = Column(DateTime, default=datetime.utcnow)  # Default to the current UTC time


In [8]:
# Create an SQLAlchemy engine with a SQLite database file named 'user_presets.db' in the 'database' folder
engine = create_engine('sqlite:///database/user_presets.db')

# Create the database tables defined in the Base class (assuming Base is a declarative_base())
# and bind them to the engine
# Base.metadata.create_all(bind=engine)

# Create a Session class bound to the engine
Session = sessionmaker(bind=engine)
def load_profile(user_name):
    session = Session()
    # Query the database to retrieve all profiles for the specified user
    profiles = session.query(Profile).filter_by(user_name=user_name).all()

    # Create a list of dictionaries containing 'user_name', 'profile_name', 'settings', and 'timestamp_UTC' for each profile
    profile_data = [
        {
            'user_name': user_name,
            'profile_name': profile.profile_name,
            'settings': profile.settings,
            'timestamp_UTC': profile.timestamp_UTC
        }
        for profile in profiles
    ]
    
    df = pd.DataFrame(profile_data)
    return df 

In [9]:
df = load_profile(user_name="Alice")
df

,user_name,profile_name,settings,timestamp_UTC
0,Alice,Alice_test_,"{'time_line': '7 Days', 'start_date': '2023-10...",2023-11-21 11:26:19.325723
1,Alice,Alice_test_2,"{'time_line': '7 Days', 'start_date': '2023-10...",2023-11-21 11:32:52.580235
2,Alice,Alice_test_3,"{'time_line': '14 Days', 'start_date': '2023-1...",2023-11-21 11:47:18.904856
3,Alice,Alice_test_5,"{'time_line': '14 Days', 'start_date': '2023-1...",2023-11-21 11:47:51.617848
4,Alice,Alice_test_6,"{'time_line': 'Lifetime', 'start_date': '2023-...",2023-11-21 11:52:12.499596
5,Alice,Alice_test_7,"{'time_line': 'Lifetime', 'start_date': '2023-...",2023-11-21 11:57:08.171335
6,Alice,Alice_test_8,"{'time_line': 'Lifetime', 'start_date': '2023-...",2023-11-21 11:59:35.881658
7,Alice,Alice_test_9,"{'time_line': 'Lifetime', 'start_date': '2023-...",2023-11-21 12:03:22.412499
8,Alice,Alice_test_x,"{'time_line': '14 Days', 'start_date': '2023-1...",2023-11-21 18:02:33.920131
